In [2]:
from npodjlEnv import NpodJLEnv
import numpy as np
env = NpodJLEnv()
env.ver()

0.3

In [3]:
def argmaxQ(Q, s, n_actions):
    if s in Q.keys():
        return(np.argmax(Q[s]))
    else:
        return(np.random.randint(0, n_actions))

In [4]:
def policy_greedy(Q, s, epsilon, n_actions):
    if np.random.rand() < epsilon:
        return(argmaxQ(Q,s,n_actions))
    else:
        return(np.random.randint(0, n_actions))

In [5]:
def SARSA(env, alpha, gamma, epsilon, Q, max_cylces):
    n_actions = len(env.actions())
    r = [None] * (2*max_cycles)
    s = [None] * (2*(max_cycles+1))
    a = [None] * (2*max_cycles)
    
    t=0
    s[t] = env.encoded_state()
    a[t] = policy_greedy(Q,s[t], epsilon, n_actions)
    obs = env.run(a[t])
    r[t] = obs['reward']
    s[t+1] = env.encoded_state() 
    if not s[t] in Q.keys():
        Q[s[t]] = [0]*n_actions
    try:
        while t < max_cycles:
            a[t+1] = policy_greedy(Q,s[t+1], epsilon, n_actions)
            obs = env.run(a[t])
            r[t+1] = obs['reward']
            s[t+2] = env.encoded_state()
            if not s[t+1] in Q.keys():
                Q[s[t+1]] = [0]*n_actions
            Q[s[t]][a[t]] = Q[s[t]][a[t]] + alpha * ( r[t] + (gamma * Q[s[t+1]][a[t+1]] ) - Q[s[t]][a[t]] )
            t += 1
            
    except KeyboardInterrupt:
        print("Press Ctrl-C to terminate while statement")
        pass
    return(s,a,r,Q)

In [21]:
Q = dict()

In [ ]:
env.reset()
alpha = 1
gamma = 0.99
epsilon = 0.5
max_cycles = 100
sol = SARSA(env, alpha, gamma, epsilon, Q, max_cycles)

In [20]:
Q

{'0.001000_50.000000_2.000000_250.000000_0.000000_0.100000': [34.47662649011227,
  -1934.5995174723905,
  0,
  0,
  0,
  597.4418850756649,
  0,
  0,
  0,
  50.90705560439528,
  0,
  0],
 '0.001000_50.000000_2.200000_250.000000_0.000000_0.100000': [0,
  0,
  0,
  0,
  0,
  0,
  -9.842611893873254e-05,
  0,
  0,
  0,
  0,
  0],
 '0.001000_50.000000_2.420000_250.000000_0.000000_0.100000': [0,
  0,
  0,
  0,
  0,
  0,
  -0.8819863473559053,
  0,
  0,
  0,
  0,
  0],
 '0.000909_50.000000_2.420000_250.000000_0.000000_0.100000': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.8927599569030917,
  0,
  0,
  0,
  0],
 '0.000826_50.000000_2.420000_250.000000_0.000000_0.100000': [0,
  0,
  0,
  0,
  -1.092146940451812,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '0.000826_50.000000_2.200000_250.000000_0.000000_0.100000': [-13.181212192323528,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.0],
 '0.000826_50.000000_2.200000_250.000000_0.000000_0.090909': [0.9830704817563856,
  0,
  0,
  0,
  0,
  0,
  0,
  0